# Falcon 9 Data Wrangling
**Author:** Muhammad Munawar Shahzad 

**Date:** August 2025  
**Project:** IBM Applied Data Science Capstone – SpaceX Falcon 9
**Author:** Muhammad Munawar Shahzad 


**Repository:** `falcon9_project`  

---

This notebook cleans and standardizes the Falcon 9 launch dataset scraped from Wikipedia.  
We’ll load the processed CSV from `data/processed/falcon_web_scraped.csv`, fix types, normalize text, engineer useful features, and save a clean, analysis-ready file.


## 📌 Project Overview
- **Input:** `data/processed/falcon_web_scraped.csv` (from Notebook 2: Web Scraping)  
- **Goal:** Produce a clean, consistent dataset for EDA, dashboards, and modeling.  
- **Output:** `data/processed/falcon_web_scraped_cleaned.csv`


## 🎯 Objectives
1. Load scraped data from the processed folder.
2. Inspect schema, missingness, and quality issues.
3. Clean and normalize:
   - Dates, strings, numeric fields.
   - Remove duplicates and whitespace/footnotes.
4. Feature engineering:
   - Year/Month from Date.
   - Extract booster/core info when available.
   - Outcome flags (success/failure) if present.
5. Save a clean, analysis-ready CSV for the next steps.


In [11]:
# -------------------------------------------------
# Imports & Project Path Bootstrap
# -------------------------------------------------
import os
from pathlib import Path
import re

import numpy as np
import pandas as pd

# Detect project root (two levels up if inside notebooks folder)
PROJECT_ROOT = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path.cwd().parents[0]

# Define data directories
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"

# Ensure folders exist
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# Define input/output file paths
INPUT_PATH = RAW_DIR / "falcon9_web_scraped.csv"         # raw folder me input
OUTPUT_PATH = PROCESSED_DIR / "falcon_web_scraped_cleaned.csv"  # processed folder me output

# Debug print paths and file existence
print(f"Project root : {PROJECT_ROOT}")
print(f"Input exists : {INPUT_PATH.exists()} - {INPUT_PATH}")
print(f"Output file  : {OUTPUT_PATH}")


Project root : d:\Projects\falcon9_project
Input exists : True - d:\Projects\falcon9_project\data\raw\falcon9_web_scraped.csv
Output file  : d:\Projects\falcon9_project\data\processed\falcon_web_scraped_cleaned.csv


## 📥 Load Data
We read the scraped CSV from the processed folder and preview the first rows.  
If column names vary (as Wikipedia tables can change), we’ll clean defensively.

In [12]:
# -------------------------------------------------
# Load the scraped dataset and quick peek
# -------------------------------------------------
df = pd.read_csv(INPUT_PATH)
print("Shape:", df.shape)
display(df.head(3))

Shape: (134, 10)


,Flight No.,Date andtime (UTC),"Version,booster[i]",Launchsite,Payload[j],Payload mass,Orbit,Customer,Launchoutcome,Boosterlanding
0,286,"January 3, 202403:44[24]",F9B5B1082‑1,"Vandenberg,SLC‑4E",Starlink:Group 7-9(22 satellites),"~16,800 kg (37,000 lb)",LEO,SpaceX,Success,Success (OCISLY)
1,287,"January 3, 202423:04[25]",F9B5B1076‑10,"Cape Canaveral,SLC‑40",Ovzon-3,"1,800 kg (4,000 lb)",GTO,Ovzon,Success,Success (LZ‑1)
2,288,"January 7, 202422:35[29]",F9B5B1067‑16,"Cape Canaveral,SLC‑40",Starlink:Group 6-35(23satellites),"~17,100 kg (37,700 lb)",LEO,SpaceX,Success,Success (ASOG)


## 🔎 Inspect Schema & Missingness
We’ll review columns, data types, basic stats, and missingness to decide cleaning steps.

In [13]:
# -------------------------------------------------
# Info, describe, and missingness
# -------------------------------------------------
df_info = df.dtypes.to_frame("dtype")
df_missing = df.isna().sum().to_frame("missing")
schema = df_info.join(df_missing)
display(schema)

# Numeric summary (only if numeric columns exist)
num_cols = df.select_dtypes(include=[np.number]).columns
if len(num_cols) > 0:
    display(df[num_cols].describe().T)
else:
    print("⚠ No numeric columns found in dataset.")

# Sample of object columns for a quick sense
obj_cols = df.select_dtypes(include="object").columns.tolist()
print("Object columns:", obj_cols[:15])

,dtype,missing
Flight No.,object,0
Date andtime (UTC),object,0
"Version,booster[i]",object,0
Launchsite,object,0
Payload[j],object,0
Payload mass,object,0
Orbit,object,0
Customer,object,0
Launchoutcome,object,0
Boosterlanding,object,0


⚠ No numeric columns found in dataset.
Object columns: ['Flight No.', 'Date andtime (UTC)', 'Version,booster[i]', 'Launchsite', 'Payload[j]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding']


## 🧼 Standardize Column Names
Wikipedia tables often include newlines, footnotes, or varying cases.  
We’ll:
- Strip whitespace and newlines
- Lowercase
- Replace spaces and special chars with underscores
- Map common names to a consistent schema (if detected)


In [14]:
# -------------------------------------------------
# Normalize column names and map common headers
# -------------------------------------------------
def normalize_col(col: str) -> str:
    col = re.sub(r"\s+", " ", str(col)).strip()
    col = col.replace("\n", " ")
    col = col.lower()
    col = re.sub(r"[^a-z0-9]+", "_", col).strip("_")
    return col

df.columns = [normalize_col(c) for c in df.columns]

# Try mapping common columns from Wikipedia launch tables to a canonical set
# This is defensive: only renames if those columns exist.
col_map_candidates = {
    "date": "date",
    "date_and_time_utc": "date",
    "launch_date": "date",
    "time_utc": "time_utc",
    "vehicle": "vehicle",
    "booster": "booster",
    "booster_core": "booster",
    "core": "booster",
    "serial": "booster",
    "launch_site": "launch_site",
    "site": "launch_site",
    "launchpad": "launch_site",
    "payload": "payload",
    "payload_mass": "payload_mass_kg",
    "payload_mass_kg": "payload_mass_kg",
    "orbit": "orbit",
    "customer": "customer",
    "mission": "mission",
    "mission_outcome": "mission_outcome",
    "landing_outcome": "landing_outcome",
    "landing_attempt": "landing_attempt",
    "landing_type": "landing_type",
    "landing_site": "landing_site",
}

df = df.rename(columns={c: col_map_candidates[c] for c in df.columns if c in col_map_candidates})

print("Columns after normalization & mapping:\n", df.columns.tolist())
display(df.head(3))


Columns after normalization & mapping:
 ['flight_no', 'date_andtime_utc', 'version_booster_i', 'launchsite', 'payload_j', 'payload_mass_kg', 'orbit', 'customer', 'launchoutcome', 'boosterlanding']


,flight_no,date_andtime_utc,version_booster_i,launchsite,payload_j,payload_mass_kg,orbit,customer,launchoutcome,boosterlanding
0,286,"January 3, 202403:44[24]",F9B5B1082‑1,"Vandenberg,SLC‑4E",Starlink:Group 7-9(22 satellites),"~16,800 kg (37,000 lb)",LEO,SpaceX,Success,Success (OCISLY)
1,287,"January 3, 202423:04[25]",F9B5B1076‑10,"Cape Canaveral,SLC‑40",Ovzon-3,"1,800 kg (4,000 lb)",GTO,Ovzon,Success,Success (LZ‑1)
2,288,"January 7, 202422:35[29]",F9B5B1067‑16,"Cape Canaveral,SLC‑40",Starlink:Group 6-35(23satellites),"~17,100 kg (37,700 lb)",LEO,SpaceX,Success,Success (ASOG)


## ✂️ Basic Text Cleaning
- Remove footnote markers like `[1]`, `[a]` etc.  
- Strip extra spaces  
- Convert obvious “N/A”, “—”, “” to proper `NaN`


In [15]:
# -------------------------------------------------
# Text normalization for object columns
# -------------------------------------------------
def clean_text(s):
    if pd.isna(s):
        return np.nan
    s = re.sub(r"\[[^\]]*\]", "", str(s))   # remove [footnotes]
    s = re.sub(r"\s+", " ", s).strip()
    if s in {"", "—", "-", "NA", "N/A", "na", "n/a"}:
        return np.nan
    return s

for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].apply(clean_text)

display(df.head(3))


,flight_no,date_andtime_utc,version_booster_i,launchsite,payload_j,payload_mass_kg,orbit,customer,launchoutcome,boosterlanding
0,286,"January 3, 202403:44",F9B5B1082‑1,"Vandenberg,SLC‑4E",Starlink:Group 7-9(22 satellites),"~16,800 kg (37,000 lb)",LEO,SpaceX,Success,Success (OCISLY)
1,287,"January 3, 202423:04",F9B5B1076‑10,"Cape Canaveral,SLC‑40",Ovzon-3,"1,800 kg (4,000 lb)",GTO,Ovzon,Success,Success (LZ‑1)
2,288,"January 7, 202422:35",F9B5B1067‑16,"Cape Canaveral,SLC‑40",Starlink:Group 6-35(23satellites),"~17,100 kg (37,700 lb)",LEO,SpaceX,Success,Success (ASOG)


## 🗓️ Dates & Types
- Parse the `date` column into `datetime` (UTC if applicable).  
- Convert numeric-looking fields (e.g., payload mass) to numeric.  
- Keep parsing errors as `NaT`/`NaN` for transparency.


In [16]:
# -------------------------------------------------
# Parse date column, numeric fields
# -------------------------------------------------
# Date parsing (defensive: only if `date` exists)
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Common numeric columns we may want
numeric_candidates = ["payload_mass_kg"]
for col in numeric_candidates:
    if col in df.columns:
        # Remove commas/units if present
        df[col] = (
            df[col]
            .astype(str)
            .str.replace(",", "", regex=False)
            .str.replace(r"[^\d\.\-]", "", regex=True)
        )
        df[col] = pd.to_numeric(df[col], errors="coerce")

display(df.head(3))


,flight_no,date_andtime_utc,version_booster_i,launchsite,payload_j,payload_mass_kg,orbit,customer,launchoutcome,boosterlanding
0,286,"January 3, 202403:44",F9B5B1082‑1,"Vandenberg,SLC‑4E",Starlink:Group 7-9(22 satellites),1.680037e+09,LEO,SpaceX,Success,Success (OCISLY)
1,287,"January 3, 202423:04",F9B5B1076‑10,"Cape Canaveral,SLC‑40",Ovzon-3,1.800400e+07,GTO,Ovzon,Success,Success (LZ‑1)
2,288,"January 7, 202422:35",F9B5B1067‑16,"Cape Canaveral,SLC‑40",Starlink:Group 6-35(23satellites),1.710038e+09,LEO,SpaceX,Success,Success (ASOG)


## 🔁 Remove Duplicates & Sort
- Drop exact duplicate rows.  
- Sort by `date` if available (chronological order helps downstream analysis).

In [17]:
# -------------------------------------------------
# Deduplicate & sort
# -------------------------------------------------
before = len(df)
df = df.drop_duplicates().copy()
after = len(df)
print(f"Removed duplicates: {before - after}")

if "date" in df.columns:
    df = df.sort_values("date").reset_index(drop=True)

display(df.head(3))

Removed duplicates: 0


,flight_no,date_andtime_utc,version_booster_i,launchsite,payload_j,payload_mass_kg,orbit,customer,launchoutcome,boosterlanding
0,286,"January 3, 202403:44",F9B5B1082‑1,"Vandenberg,SLC‑4E",Starlink:Group 7-9(22 satellites),1.680037e+09,LEO,SpaceX,Success,Success (OCISLY)
1,287,"January 3, 202423:04",F9B5B1076‑10,"Cape Canaveral,SLC‑40",Ovzon-3,1.800400e+07,GTO,Ovzon,Success,Success (LZ‑1)
2,288,"January 7, 202422:35",F9B5B1067‑16,"Cape Canaveral,SLC‑40",Starlink:Group 6-35(23satellites),1.710038e+09,LEO,SpaceX,Success,Success (ASOG)


## 🧩 Feature Engineering
We’ll add helpful fields for analysis:
- **year** and **month** from `date`  
- **success flags** from `mission_outcome` and/or `landing_outcome` (if present)  
- Light **booster extraction** (e.g., first code token) where available

In [18]:
# -------------------------------------------------
# Feature Engineering
# -------------------------------------------------
# Year / Month
if "date" in df.columns:
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month

# Success flags from mission_outcome
def outcome_success_flag(x):
    if pd.isna(x):
        return np.nan
    s = str(x).lower()
    # Common patterns on Wikipedia: "Success", "Partial failure", "Failure", etc.
    if "success" in s:
        return 1
    if "failure" in s:
        return 0
    return np.nan

if "mission_outcome" in df.columns:
    df["mission_success"] = df["mission_outcome"].apply(outcome_success_flag)

if "landing_outcome" in df.columns:
    df["landing_success"] = df["landing_outcome"].apply(outcome_success_flag)

# Booster code extraction (very heuristic)
if "booster" in df.columns:
    df["booster_clean"] = df["booster"].str.extract(r"([A-Za-z]*\d+)")

## ✅ Final Sanity Checks
- Re-inspect schema  
- Check missingness again  
- Quick counts by year (if date exists)

In [19]:
# -------------------------------------------------
# Sanity checks: schema, missingness, quick pivots
# -------------------------------------------------
schema_after = df.dtypes.to_frame("dtype").join(df.isna().sum().to_frame("missing"))
display(schema_after)

if "year" in df.columns:
    year_counts = df.groupby("year").size().reset_index(name="rows")
    display(year_counts.sort_values("year"))

,dtype,missing
flight_no,object,0
date_andtime_utc,object,0
version_booster_i,object,0
launchsite,object,0
payload_j,object,0
payload_mass_kg,float64,9
orbit,object,0
customer,object,0
launchoutcome,object,0
boosterlanding,object,0


## 💾 Save Cleaned Dataset
We save the cleaned, analysis-ready dataset to:



This file will be the input for:
- **4_exploratory_analysis.ipynb** (EDA & visualizations)  
- **5_dash_dashboard.ipynb** (Dash app)  
- **Predictive modeling** steps later

In [20]:
# -------------------------------------------------
# Save cleaned dataset
# -------------------------------------------------
df.to_csv(OUTPUT_PATH, index=False)
print(f"✅ Cleaned data saved to: {OUTPUT_PATH}")
print("Rows:", len(df))


✅ Cleaned data saved to: d:\Projects\falcon9_project\data\processed\falcon_web_scraped_cleaned.csv
Rows: 134


## 📋 Summary
- Loaded scraped data from `data/processed/falcon_web_scraped.csv`  
- Standardized column names & cleaned text/footnotes  
- Parsed dates & numeric fields, removed duplicates  
- Engineered features: year/month, success flags, booster extraction  
- Saved clean dataset → `data/processed/falcon_web_scraped_cleaned.csv`

## 🚀 Next Steps
Proceed to **`4_eda_and_visualization_
with_sql.ipynb`**:
- Descriptive statistics and distributions
- Trend lines by year/month
- Outcome analysis (mission vs. landing success)
- Visuals (Plotly/Folium in later steps)

## 📚 References
- SpaceX Wikipedia launch lists (Falcon 9 / Falcon Heavy)  
- Pandas User Guide: https://pandas.pydata.org/docs/  
- Regular Expressions (Python `re`): https://docs.python.org/3/library/re.html